# Tema 6 - Practica 1: Redis

In [2]:
import redis
import json

# Apartado A

Configura la conexión a la BBDD Redis que previamente has levantado en Docker.

In [7]:
# Configura la conexión
redis_host = "localhost" # Cambia esto por la IP o el hostname del contenedor si no está en localhost
redis_port = 6379 # Puerto por defecto
redis_db = 0 # BBDD a usar dentro de redis (Por defecto de 0 a 15)

In [8]:
# Crear conexion
redis_client = redis.StrictRedis(host=redis_host, port=redis_port, db=redis_db, decode_responses=True)

# Apartado B

Configura un metodo "crear_carrito" que cree un carrito de la compra con la cesta vacía, un username vinculado y un importe total de 0.

In [9]:
# Version Clave - Valor

def crear_carrito_v1(username):
    cart_data = {
        "username": username,
        "cesta": [],
        "importe_total": 0.0,
    }

    redis_client.set(username, json.dumps(cart_data))

In [10]:
crear_carrito_v1("jorgegc_v1")

In [11]:
# Version con estructura de datos hashes de Redis

def crear_carrito_v2(username):
    redis_client.hset(username, 
    mapping={
        "username": username,
        "cesta": json.dumps([]),
        "importe_total": 0.0
        })

In [12]:
crear_carrito_v2("jorgegc_v2")

Las ventajas de usar hashes para este caso son:
* Optimizacion del espacio: Redis almacena hashes de manera compacta.
* Acceso granular: No necesitamos acceder a toda la estructura de datos para modificar, por ejemplo, solo el importe total.
* Escalabilidad: Mejor en el caso de gestionar muchos usuarios y carritos.

# Apartado C

Crear una funcion que permita recuperar el contenido del carrito

In [18]:
def recuperar_carrito_v1(username):
    cart_json = redis_client.get(username)
    return json.loads(cart_json) if cart_json else None

In [19]:
recuperar_carrito_v1("jorgegc_v1")

{'username': 'jorgegc_v1', 'cesta': [], 'importe_total': 0.0}

In [13]:
def recuperar_carrito_v2(username):
    # Esto nos da todo (username, cesta y importe total) pero esta todo en string
    cart_hash = redis_client.hgetall(username)
    if not cart_hash:
        return None

    # Reconstruir la estructura del carrito
    cart_hash["cesta"] = json.loads(cart_hash["cesta"]) if "cesta" in cart_hash else []
    cart_hash["importe_total"] = float(cart_hash["importe_total"])
    return cart_hash

In [14]:
recuperar_carrito_v2("jorgegc_v2")

{'username': 'jorgegc_v2', 'cesta': [], 'importe_total': 0.0}

# Apartado D

Funcion que calcule el importe total dada una cesta de la compra

In [15]:
def calcular_total(cesta):
    return sum(item["cantidad"] * item["precio_unitario"] for item in cesta)

In [20]:
carrito = recuperar_carrito_v1("jorgegc_v1")
carrito["importe_total"] = calcular_total(carrito["cesta"])
carrito

{'username': 'jorgegc_v1', 'cesta': [], 'importe_total': 0}

In [16]:
carrito = recuperar_carrito_v2("jorgegc_v2")
carrito["importe_total"] = calcular_total(carrito["cesta"])
carrito

{'username': 'jorgegc_v2', 'cesta': [], 'importe_total': 0}

# Apartado E

Crear una funcion que guarde el carrito de la compra en la BBDD

In [21]:
def guardar_carrito_v1(username, carrito):
    redis_client.set(username, json.dumps(carrito))

In [17]:
def guardar_carrito_v2(username, carrito):
    redis_client.hset(username, 
        mapping={
            "username": username,
            "cesta": json.dumps(carrito["cesta"]),
            "importe_total": carrito["importe_total"]
        })

# Apartado F

Crear una funcion que permita añadir un item al carrito

In [22]:
def añadir_item(username, item):
    carrito = recuperar_carrito_v1(username)
    #carrito = recuperar_carrito_v2(username)
    carrito["cesta"].append(item)
    carrito["importe_total"] = calcular_total(carrito["cesta"])
    guardar_carrito_v1(username, carrito)

In [36]:
item_1 = {"articulo": "item_1", "cantidad": 2, "precio_unitario": 2}
item_2 = {"articulo": "item_2", "cantidad": 1, "precio_unitario": 15}

añadir_item("jorgegc_v1", item_1)
añadir_item("jorgegc_v1", item_2)

In [37]:
recuperar_carrito_v1("jorgegc_v1")

{'username': 'jorgegc_v1',
 'cesta': [{'articulo': 'item_1', 'cantidad': 2, 'precio_unitario': 2},
  {'articulo': 'item_2', 'cantidad': 1, 'precio_unitario': 15}],
 'importe_total': 19}

# Apartado G

Crear una funcion que permita eliminar un item del carrito

In [38]:
def eliminar_item(username, item_name):
    carrito = recuperar_carrito_v1(username)
    carrito["cesta"] = [item for item in carrito["cesta"] if item["articulo"] != item_name]
    carrito["importe_total"] = calcular_total(carrito["cesta"])
    guardar_carrito_v1(username, carrito)

In [39]:
eliminar_item("jorgegc_v1", "item_1")

recuperar_carrito_v1("jorgegc_v1")

{'username': 'jorgegc_v1',
 'cesta': [{'articulo': 'item_2', 'cantidad': 1, 'precio_unitario': 15}],
 'importe_total': 15}

# Apartado H

Crear una funcion que permita vaciar completamente el carrito

In [40]:
def vaciar_carrito(username):
    crear_carrito_v1(username)

In [41]:
vaciar_carrito("jorgegc_v1")

recuperar_carrito_v1("jorgegc_v1")

{'username': 'jorgegc_v1', 'cesta': [], 'importe_total': 0.0}